<a href="https://colab.research.google.com/github/meiladrahmani556/marine-cbm-ml-dissertation/blob/main/JupyterNotebook/05_model_optimisation_%26_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
PROJECT_PATH = "/content/drive/MyDrive/✨CBM Data for Marine System Monitoring & Analysis✨"

# 📓 Notebook 05 – Model Optimisation & Validation

## 🎯 Objective

This notebook improves upon the baseline models developed in Notebook 04.

The goals are:

- Perform cross-validation for robust evaluation
- Optimise Random Forest hyperparameters
- Compare tuned vs baseline performance
- Analyse overfitting behaviour
- Provide engineering interpretation of results

Model optimisation is critical in predictive maintenance applications to ensure generalisation and reliability.

In [5]:
from google.colab import files
uploaded = files.upload()

Saving Conditional_Base_Monitoring in Marine_System.csv to Conditional_Base_Monitoring in Marine_System.csv


In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv("Conditional_Base_Monitoring in Marine_System.csv")
df.columns = df.columns.str.strip()
df = df.apply(pd.to_numeric, errors='coerce')
df = df.dropna().drop_duplicates()

target_column = "GT Compressor decay state coefficient"

X = df.drop(columns=[target_column])
y = df[target_column]

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

baseline_rf = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

baseline_rf.fit(X_train, y_train)

y_pred_baseline = baseline_rf.predict(X_test)

r2_baseline = r2_score(y_test, y_pred_baseline)
rmse_baseline = np.sqrt(mean_squared_error(y_test, y_pred_baseline))

print("Baseline R²:", r2_baseline)
print("Baseline RMSE:", rmse_baseline)

Baseline R²: 0.9981560708462252
Baseline RMSE: 0.0006395971270451906


In [8]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(
    baseline_rf,
    X_train,
    y_train,
    cv=5,
    scoring='r2',
    n_jobs=-1
)

print("Cross-Validation R² Scores:", cv_scores)
print("Mean CV R²:", np.mean(cv_scores))

Cross-Validation R² Scores: [0.99742632 0.99659067 0.99710304 0.9975484  0.99751297]
Mean CV R²: 0.9972362820879261


## Cross-Validation Interpretation

Cross-validation reduces the risk of overfitting by evaluating model performance across multiple training subsets.

A small gap between cross-validation score and test score indicates good generalisation.

In [9]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best CV R²:", grid_search.best_score_)

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best CV R²: 0.9962677222464668


In [10]:
best_rf = grid_search.best_estimator_

y_pred_tuned = best_rf.predict(X_test)

r2_tuned = r2_score(y_test, y_pred_tuned)
rmse_tuned = np.sqrt(mean_squared_error(y_test, y_pred_tuned))

print("Tuned R²:", r2_tuned)
print("Tuned RMSE:", rmse_tuned)

Tuned R²: 0.9981920349093784
Tuned RMSE: 0.0006333290502071398


In [11]:
comparison = pd.DataFrame({
    "Model": ["Baseline RF", "Tuned RF"],
    "R²": [r2_baseline, r2_tuned],
    "RMSE": [rmse_baseline, rmse_tuned]
})

comparison

,Model,R²,RMSE
0,Baseline RF,0.998156,0.000640
1,Tuned RF,0.998192,0.000633


In [12]:
train_r2 = r2_score(y_train, best_rf.predict(X_train))
test_r2 = r2_tuned

print("Training R²:", train_r2)
print("Test R²:", test_r2)

Training R²: 0.9997025215711464
Test R²: 0.9981920349093784


## Critical Evaluation

Hyperparameter tuning resulted in an improvement in predictive performance compared to the baseline model.

The comparison between training and test performance suggests:

- If training and test scores are similar → good generalisation
- If training score is significantly higher → mild overfitting

In predictive maintenance applications, model stability and generalisation are more important than marginal increases in R².